# Task

Find the most popular female name on a ship. The dataset was taken from [kaggle.com](https://www.kaggle.com/c/titanic/data).

In [72]:
import pandas as pd
df=pd.read_csv('titanic.csv', index_col='PassengerId')
df

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S


Female name migh be extracted easily thanks to "sex" column

In [73]:
df_fem=df.loc[df.Sex=='female'].Name
df_fem=df_fem.to_frame()
df_fem.head(10)

,Name
PassengerId,
2,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
3,"Heikkinen, Miss. Laina"
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
9,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"
10,"Nasser, Mrs. Nicholas (Adele Achem)"
11,"Sandstrom, Miss. Marguerite Rut"
12,"Bonnell, Miss. Elizabeth"
15,"Vestrom, Miss. Hulda Amanda Adolfina"
16,"Hewlett, Mrs. (Mary D Kingcome)"


There are 2 common cases where exactly first names are seen:<br>
– the word that follows "Miss."<br>
– the word that follows "(" sign

In [74]:
#detecting rows which have ( sign
#those which do not have are shown as []
import re
df_fem_Names1 = df_fem['Name'].apply(lambda x: re.findall('\((.*?)\)',x))
df_fem_Names1=df_fem_Names1.to_frame()

#replacing [] with NA
df_fem_Names1 = df_fem_Names1['Name'].str.get(0)
df_fem_Names1=df_fem_Names1.to_frame()

#extracting only first names
df_fem_Names1=df_fem_Names1.dropna()
df_fem_Names1['OnlyNames1'] =  df_fem_Names1['Name'].apply(lambda x: x.split()[0])
df_fem_Names1

,Name,OnlyNames1
PassengerId,,
2,Florence Briggs Thayer,Florence
4,Lily May Peel,Lily
9,Elisabeth Vilhelmina Berg,Elisabeth
10,Adele Achem,Adele
16,Mary D Kingcome,Mary
...,...,...
872,Sallie Monypeny,Sallie
875,Hannah Wizosky,Hannah
880,Lily Alexenia Wilson,Lily


In [75]:
#list of names where the word "Miss." was found
lst=df_fem['Name'].to_list()
new_lst=[]
for i in lst:
    keyword = 'Miss.'
    before_keyword, keyword, after_keyword = i.partition(keyword)
    new_lst.append(after_keyword)
new_lst[0:10]

['',
 ' Laina',
 '',
 '',
 '',
 ' Marguerite Rut',
 ' Elizabeth',
 ' Hulda Amanda Adolfina',
 '',
 '']

In [76]:
#list to dataframe
df_fem_Names2=pd.DataFrame(new_lst)
df_fem_Names2=df_fem_Names2.rename(columns={0: 'Name'})

#extracting only first names
nan_value = float("NaN")
df_fem_Names2.replace("", nan_value, inplace=True)
df_fem_Names2=df_fem_Names2.dropna()
df_fem_Names2['OnlyNames2'] = df_fem_Names2['Name'].apply(lambda x: x.split()[0])
df_fem_Names2

,Name,OnlyNames2
1,Laina,Laina
5,Marguerite Rut,Marguerite
6,Elizabeth,Elizabeth
7,Hulda Amanda Adolfina,Hulda
11,"Anna ""Annie""",Anna
...,...,...
304,Asuncion,Asuncion
307,"Adele Kiamie ""Jane""",Adele
310,Gerda Ulrika,Gerda
312,Margaret Edith,Margaret


In [77]:
#resulting dataframe with all female first names
df_fem_f=pd.concat([df_fem_Names1['OnlyNames1'], df_fem_Names2['OnlyNames2']], axis=0)
df_fem_f=df_fem_f.to_frame()
df_fem_f=df_fem_f.rename(columns={0:'OnlyNames'})
df_fem_f

,OnlyNames
2,Florence
4,Lily
9,Elisabeth
10,Adele
16,Mary
...,...
304,Asuncion
307,Adele
310,Gerda
312,Margaret


In [78]:
print('Top-3 popular female names on a ship are:')
df_fem_f.OnlyNames.value_counts().head(3)

Top-3 popular female names on a ship are:


Anna         15
Mary         14
Elizabeth    11
Name: OnlyNames, dtype: int64